In [2]:
# Filter data on categorical feature and show table and plot
#https://gist.github.com/dennisobrien/450d7da20daaba6d39d0

import bokeh.embed
import bokeh.io
import bokeh.models
import bokeh.models.widgets
import bokeh.plotting
import pandas as pd
import pandas as pd
import numpy as np
from bokeh.plotting import figure,output_notebook,show,output_file
from bokeh.models import ColumnDataSource, Select, TableColumn, DataTable
from bokeh.layouts import column

output_file("data_table.html")

mean, cov = [0, 1], [(1, .5), (.5, 1)]
data = np.random.multivariate_normal(mean, cov, 200)
df = pd.DataFrame(data, columns=["x", "y"])

df['country'] = ['A' if x > .4 else 'B' if  x > .1 else 'C' for x in df['x']]
df['year'] = df['x']
df['NY.GDP.PCAP.KD'] = df['y']

df = df.reset_index()

source = ColumnDataSource(df)
original_source = ColumnDataSource(df)
columns = [
    TableColumn(field="country", title="Country"),
    TableColumn(field="year", title="Year"),
    TableColumn(field="NY.GDP.PCAP.KD", title="NY.GDP.PCAP.KD"),
]
data_table = DataTable(source=source, columns=columns)

# callback code to be used by all the filter widgets
# requires (source, original_source, country_select_obj, year_select_obj, target_object)
combined_callback_code = """
var data = source.get('data');
var original_data = original_source.get('data');
var country = country_select_obj.get('value');
console.log("country: " + country);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['country'].length; ++i) {
        if (( country === 'ALL' || original_data['country'][i] === country)) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.trigger('change');
source.trigger('change');
"""

# define the filter widgets, without callbacks for now
#country_list =  df['country'].unique().tolist()
#country_select = Select(title="Country:", value=country_list[0], options=country_list)
country_select = Select(title="Country:", value='ALL', options=['ALL','A','B','C'])


#year_list = ['ALL'] + df['year'].unique().tolist()
#year_select = bokeh.models.widgets.Select(title="Year:", value=year_list[0], options=year_list)

# now define the callback objects now that the filter widgets exist
generic_callback = bokeh.models.CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              country_select_obj=country_select, 
#              year_select_obj=year_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
country_select.callback = generic_callback
#year_select.callback = generic_callback

#---
p1 = figure(plot_width=400, plot_height=400)
p1.circle('NY.GDP.PCAP.KD', 'year', source=source, alpha=0.6)
#---

p = column(country_select, data_table,p1)
show(p)

In [22]:
data_table.view

CDSView(id='519765b8-3944-41b7-8c4b-a2caac2b61bc', ...)

In [24]:
from datetime import date
from random import randint

from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_file, show

output_file("data_table.html")

data = dict(
        dates=[date(2014, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="dates", title="Date", formatter=DateFormatter()),
        TableColumn(field="downloads", title="Downloads"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

show((data_table))